In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


In [16]:
input_image = torch.rand(3,28,28) #creating 3 images of 28*28
print(input_image.size())

torch.Size([3, 28, 28])


In [18]:
flatten = nn.Flatten() #turns 28*28 into 784
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [20]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0403, -0.0370,  0.3276, -0.0755,  0.0405,  0.3260,  0.1724,  0.3359,
         -0.1543, -0.0811, -0.1313, -0.2377,  0.0498,  0.2627, -0.5232, -0.2298,
         -0.5627, -0.0177, -0.1018, -0.4485],
        [-0.0252, -0.0974, -0.1181,  0.0114,  0.0209,  0.0106,  0.3358,  0.2102,
         -0.0655, -0.1612, -0.1206,  0.1699, -0.3109,  0.1621, -0.4082, -0.3897,
         -0.3306,  0.0043,  0.0534, -0.3532],
        [ 0.0377,  0.2443,  0.2328, -0.0046, -0.3618,  0.3019,  0.2841,  0.2431,
         -0.3005, -0.3177, -0.2090, -0.1348, -0.3586, -0.1239, -0.3499, -0.0079,
         -0.1853,  0.1517,  0.1562, -0.4008]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.3276, 0.0000, 0.0405, 0.3260, 0.1724, 0.3359, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0498, 0.2627, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0114, 0.0209, 0.0106, 0.3358, 0.2102, 0.0000,
         0.0000, 0.0000, 0.1699, 0.0000, 0.1621, 0.00

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0309, -0.0278,  0.0234,  ...,  0.0336,  0.0134, -0.0354],
        [ 0.0341,  0.0147,  0.0189,  ...,  0.0168,  0.0121,  0.0173]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0018,  0.0068], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0385,  0.0334,  0.0440,  ..., -0.0092, -0.0161, -0.0105],
        [ 0.0223,  0.0247, -0.0030,  ...,  0.0148, -0.0301,  0.0010]],
       device='cuda:0', grad_fn=<Sl